In [1]:
from tools.data import get_datasets

train, test = get_datasets()

Reading data file


In [2]:
%matplotlib inline

import matplotlib.pyplot as plt

In [3]:
# Classification -1 vs >=0
# on prépare les données:
# tout ce qui est un chiffre -> 1
# tout ce qui est du bruit -> 0
import numpy as np

train[train[:,-1] >= 0,-1] =  0
test[test[:,-1] >= 0,-1] =  0

train[:,-1] = train[:,-1] + 1
test[:,-1] = test[:,-1] + 1

In [49]:
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation, Merge, Flatten
from keras.optimizers import SGD, Adadelta
from keras.layers.convolutional import Convolution1D, MaxPooling1D

nb_filters = 10
filter_length = 5

first_layers = []
for i in range(4):
    first_layers.append(Sequential())
    first_layers[-1].add(Convolution1D(nb_filters, filter_length,
                        border_mode='valid',
                        input_shape=(408, 1), init='glorot_normal'))
    first_layers[-1].add(MaxPooling1D(pool_length=2))
    first_layers[-1].add(Activation('tanh'))
    first_layers[-1].add(Convolution1D(nb_filters, filter_length,
                        border_mode='valid', init='glorot_normal'))
    first_layers[-1].add(MaxPooling1D(pool_length=2))
    first_layers[-1].add(Activation('tanh'))
    first_layers[-1].add(Flatten())


model = Sequential()
model.add(Merge(first_layers, mode='concat', concat_axis=1))

model.add(Dense(100))
model.add(Activation('tanh'))
model.add(Dense(2))
model.add(Activation('softmax'))

adadelta = Adadelta()
model.compile(loss='categorical_crossentropy', optimizer=adadelta)

In [37]:
from keras.utils import np_utils

train_x = train.astype('float32')
valid_x = test[:5000].astype('float32')
test_x = test[5000:].astype('float32')

train_y = np_utils.to_categorical(train[:,-1], 2)
valid_y = np_utils.to_categorical(test[:5000,-1], 2)
test_y = np_utils.to_categorical(test[5000:,-1], 2)

train_4 = [train_x[:,1+512*0:1+512*0+408,None], train_x[:,1+512*1:1+512*1+408,None], train_x[:,1+512*2:1+512*2+408,None], train_x[:,1+512*3:1+512*3+408,None]]
valid_4 = [valid_x[:,1+512*0:1+512*0+408,None], valid_x[:,1+512*1:1+512*1+408,None], valid_x[:,1+512*2:1+512*2+408,None], valid_x[:,1+512*3:1+512*3+408,None]]
test_4 = [test_x[:,1+512*0:1+512*0+408,None], test_x[:,1+512*1:1+512*1+408,None], test_x[:,1+512*2:1+512*2+408,None], test_x[:,1+512*3:1+512*3+408,None]]


In [50]:
batch_size = 250
nb_epoch = 100

model.fit(train_4, train_y,
          batch_size=batch_size, nb_epoch=nb_epoch,
          show_accuracy=True, verbose=2,
          validation_data=(valid_4, valid_y))
score = model.evaluate(test_4, test_y,
                       show_accuracy=True, verbose=0)
print score

Train on 30000 samples, validate on 5000 samples
Epoch 1/100
33s - loss: 0.9420 - acc: 0.7276 - val_loss: 0.5821 - val_acc: 0.7384
Epoch 2/100
33s - loss: 0.5984 - acc: 0.7276 - val_loss: 0.5905 - val_acc: 0.7384
Epoch 3/100
33s - loss: 0.5981 - acc: 0.7277 - val_loss: 0.6061 - val_acc: 0.7388
Epoch 4/100
33s - loss: 0.5976 - acc: 0.7284 - val_loss: 0.5943 - val_acc: 0.7386
Epoch 5/100
33s - loss: 0.6021 - acc: 0.7279 - val_loss: 0.5735 - val_acc: 0.7390
Epoch 6/100
33s - loss: 0.6083 - acc: 0.7240 - val_loss: 0.5917 - val_acc: 0.7388
Epoch 7/100
33s - loss: 0.6044 - acc: 0.7279 - val_loss: 0.6010 - val_acc: 0.7390
Epoch 8/100
33s - loss: 0.5985 - acc: 0.7279 - val_loss: 0.6194 - val_acc: 0.7388
Epoch 9/100
33s - loss: 0.6015 - acc: 0.7282 - val_loss: 0.5896 - val_acc: 0.7388
Epoch 10/100
33s - loss: 0.6003 - acc: 0.7238 - val_loss: 0.5776 - val_acc: 0.7392
Epoch 11/100
33s - loss: 0.6020 - acc: 0.7202 - val_loss: 0.5747 - val_acc: 0.7390
Epoch 12/100
33s - loss: 0.5931 - acc: 0.7239 -

In [47]:
print score
model.fit(train_4, train_y,
          batch_size=batch_size, nb_epoch=nb_epoch,
          show_accuracy=True, verbose=2,
          validation_data=(valid_4, valid_y))
score = model.evaluate(test_4, test_y,
                       show_accuracy=True, verbose=0)

[0.61743914956494994, 0.73006852749456796]
Train on 30000 samples, validate on 5000 samples
Epoch 1/100
32s - loss: 0.6286 - acc: 0.7186 - val_loss: 0.6145 - val_acc: 0.7396
Epoch 2/100
32s - loss: 0.6015 - acc: 0.7253 - val_loss: 0.5728 - val_acc: 0.7408
Epoch 3/100
32s - loss: 0.6172 - acc: 0.7217 - val_loss: 0.5834 - val_acc: 0.7404
Epoch 4/100
32s - loss: 0.6166 - acc: 0.7224 - val_loss: 0.5868 - val_acc: 0.7408
Epoch 5/100
32s - loss: 0.6079 - acc: 0.7302 - val_loss: 0.5746 - val_acc: 0.7408
Epoch 6/100
32s - loss: 0.5953 - acc: 0.7304 - val_loss: 0.5898 - val_acc: 0.7408
Epoch 7/100
32s - loss: 0.6254 - acc: 0.7178 - val_loss: 1.5663 - val_acc: 0.7384
Epoch 8/100
32s - loss: 1.2727 - acc: 0.5930 - val_loss: 0.8187 - val_acc: 0.7386
Epoch 9/100
32s - loss: 1.3237 - acc: 0.5839 - val_loss: 0.5947 - val_acc: 0.7392
Epoch 10/100
33s - loss: 1.2637 - acc: 0.6048 - val_loss: 2.3780 - val_acc: 0.7384
Epoch 11/100
32s - loss: 1.3935 - acc: 0.5946 - val_loss: 0.8742 - val_acc: 0.2614
Epoc

KeyboardInterrupt: 

In [ ]:
# ca c'est le résultat qu'on obtient en prenant un classifieur
# qui prédit tout le temps 1 (classe majoritaire)
print float(np.sum(test[:,-1]==1))/test.shape[0]